# API Reference

> Complete reference for all FastStripe methods and classes

This reference provides detailed information about all FastStripe classes, methods, and their parameters.

In [ ]:
from faststripe.core import StripeApi
import os

## StripeApi Class

The main class for interacting with the Stripe API.

In [ ]:
# Initialize StripeApi
sapi = StripeApi(
    api_key='sk_test_...', # Your Stripe secret key
    base_url='https://api.stripe.com'  # Optional, defaults to Stripe's API
)

### Constructor Parameters

- **api_key** (str): Your Stripe secret API key
- **base_url** (str, optional): Base URL for Stripe API. Defaults to 'https://api.stripe.com'

### Available Resource Groups

All Stripe resources are organized into logical groups:

In [ ]:
# Example resource groups available on StripeApi instance
resource_groups = [
    'account',           # Account information
    'customers',         # Customer management
    'products',          # Product catalog
    'prices',           # Pricing information
    'checkout_sessions', # Checkout sessions
    'payment_intents',   # Payment intents
    'payment_methods',   # Payment methods
    'subscriptions',     # Subscription management
    'invoices',         # Invoice management
    'coupons',          # Discount coupons
    'webhook_endpoints', # Webhook configuration
    # ... and many more
]

# Each group has create/fetch methods
print("Customer methods:")
print(dir(sapi.customers))

## High-Level Convenience Methods

FastStripe provides simplified methods for common workflows.

### one_time_payment()

Create a one-time payment checkout session.

In [ ]:
checkout = sapi.one_time_payment(
    product_name='My Product',       # Product name
    amount_cents=2000,              # Amount in cents ($20.00)
    success_url='https://example.com/success',  # Success redirect URL
    cancel_url='https://example.com/cancel',    # Cancel redirect URL
    currency='usd'                   # Currency code (optional, defaults to 'usd')
)

print(f"Payment URL: {checkout.url}")

**Parameters:**
- **product_name** (str): Name of the product
- **amount_cents** (int): Price in cents
- **success_url** (str): URL to redirect after successful payment
- **cancel_url** (str): URL to redirect if payment is cancelled
- **currency** (str, optional): Currency code. Defaults to 'usd'

**Returns:** Checkout session object with payment URL

### subscription()

Create a recurring subscription checkout session.

In [ ]:
subscription = sapi.subscription(
    product_name='Monthly Plan',     # Product name
    amount_cents=999,               # Amount in cents ($9.99)
    success_url='https://example.com/welcome',  # Success redirect URL
    cancel_url='https://example.com/pricing',   # Cancel redirect URL
    currency='usd',                 # Currency code (optional)
    interval='month',               # Billing interval (optional)
    customer_email='user@example.com'  # Pre-fill email (optional)
)

print(f"Subscription URL: {subscription.url}")

**Parameters:**
- **product_name** (str): Name of the subscription product
- **amount_cents** (int): Price in cents per billing period
- **success_url** (str): URL to redirect after successful subscription
- **cancel_url** (str): URL to redirect if subscription is cancelled
- **currency** (str, optional): Currency code. Defaults to 'usd'
- **interval** (str, optional): Billing interval ('month', 'year', etc.). Defaults to 'month'
- **customer_email** (str, optional): Pre-fill customer email in checkout

**Returns:** Checkout session object with subscription URL

### find_product()

Find a product by name.

In [ ]:
product = sapi.find_product('My Product')
if product:
    print(f"Found product: {product.id}")
else:
    print("Product not found")

**Parameters:**
- **name** (str): Product name to search for

**Returns:** Product object if found, None otherwise

### find_prices()

Find all prices for a product.

In [ ]:
prices = sapi.find_prices('prod_abc123')
print(f"Found {len(prices)} prices for product")

**Parameters:**
- **product_id** (str): Stripe product ID

**Returns:** List of price objects

### priced_product()

Create or find a product with associated price.

In [ ]:
product, price = sapi.priced_product(
    product_name='My Product',
    amount_cents=2000,
    currency='usd',
    recurring={'interval': 'month'}  # Optional for subscriptions
)

print(f"Product: {product.id}, Price: {price.id}")

**Parameters:**
- **product_name** (str): Product name
- **amount_cents** (int): Price in cents
- **currency** (str, optional): Currency code. Defaults to 'usd'
- **recurring** (dict, optional): Recurring billing configuration

**Returns:** Tuple of (product, price) objects

## Common Resource Methods

Each resource group provides consistent `create` and `fetch` methods.

### create() methods

Create new resources (corresponds to POST endpoints).

In [ ]:
# Create a customer
customer = sapi.customers.create(
    email='user@example.com',
    name='John Doe',
    metadata={'user_id': '12345'}
)

# Create a product
product = sapi.products.create(
    name='My Product',
    description='A great product'
)

# Create a price
price = sapi.prices.create(
    product=product.id,
    unit_amount=2000,
    currency='usd'
)

### fetch() methods

Retrieve existing resources (corresponds to GET endpoints).

In [ ]:
# Fetch a specific customer
customer = sapi.customers.fetch('cus_abc123')

# Fetch multiple customers with filters
customers = sapi.customers.fetch(
    limit=10,
    email='user@example.com'
)

# Fetch products
products = sapi.products.fetch(limit=20, active=True)

### update() methods

Update existing resources (available on some resources).

In [ ]:
# Update customer
updated_customer = sapi.customers.update(
    'cus_abc123',
    description='VIP Customer',
    metadata={'tier': 'premium'}
)

# Update subscription
updated_subscription = sapi.subscriptions.update(
    'sub_abc123',
    cancel_at_period_end=True
)

## Response Objects

All API responses are returned as AttrDict objects, allowing both dictionary and dot notation access.

In [ ]:
customer = sapi.customers.create(email='user@example.com', name='John Doe')

# Dictionary-style access
print(customer['id'])
print(customer['email'])

# Dot notation access (preferred)
print(customer.id)
print(customer.email)
print(customer.created)  # Unix timestamp

## Error Handling

FastStripe uses the underlying httpx library for HTTP requests. Stripe API errors are returned as JSON responses.

In [ ]:
try:
    # This will fail with invalid customer ID
    customer = sapi.customers.fetch('invalid_id')
except Exception as e:
    print(f"Error: {e}")
    # Handle error appropriately

## Parameter Flattening

FastStripe automatically flattens nested parameters for Stripe's form-encoded format.

In [ ]:
# Nested parameters are automatically flattened
customer = sapi.customers.create(
    email='user@example.com',
    address={
        'line1': '123 Main St',
        'city': 'San Francisco',
        'state': 'CA',
        'postal_code': '94105',
        'country': 'US'
    },
    metadata={
        'user_id': '12345',
        'plan': 'premium'
    }
)

# FastStripe converts this to:
# {
#     'email': 'user@example.com',
#     'address[line1]': '123 Main St',
#     'address[city]': 'San Francisco',
#     'address[state]': 'CA',
#     'address[postal_code]': '94105',
#     'address[country]': 'US',
#     'metadata[user_id]': '12345',
#     'metadata[plan]': 'premium'
# }

## Method Documentation

All methods include comprehensive documentation accessible via your IDE or the `?` operator in Jupyter.

In [ ]:
# View method documentation
sapi.customers.create?

## Environment Variables

Common environment variables used with FastStripe:

- `STRIPE_SECRET_KEY`: Your Stripe secret API key
- `STRIPE_WEBHOOK_SECRET`: Webhook endpoint secret for signature verification
- `STRIPE_PUBLISHABLE_KEY`: Your Stripe publishable key (for frontend use)

Example `.env` file:
```
STRIPE_SECRET_KEY=sk_test_...
STRIPE_WEBHOOK_SECRET=whsec_...
STRIPE_PUBLISHABLE_KEY=pk_test_...
```